In [1]:
import math
from collections import defaultdict
from dataclasses import dataclass
from typing import Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset


In [2]:
@dataclass
class TrainingConfig:
    sparsity_coeff: float = 0.1  # Reduced from 1.0
    temp_start: float = 2.0  # Start with higher temperature
    temp_end: float = 0.01  # End with lower temperature
    epochs: int = 100
    num_examples: int = 1000
    batch_size: int = 32
    learning_rate: float = 0.001  # Reduced from 0.01
    num_classes: int = 4
    device: str = "mps"


class CosineScheduler:
    def __init__(self, start_value: float, end_value: float, num_steps: int):
        self.start = start_value
        self.end = end_value
        self.num_steps = num_steps
        self.current_step = 0
        self.value = start_value

    def step(self) -> float:
        value = self.end + 0.5 * (self.start - self.end) * (
            1 + math.cos((self.current_step / self.num_steps) * math.pi)
        )
        self.current_step = min(self.current_step + 1, self.num_steps)
        self.value = value
        return value


class MetricsTracker:
    def __init__(self):
        self.metrics = defaultdict(list)

    def update_epoch_metrics(self, epoch_metrics: Dict[str, float]):
        for key, value in epoch_metrics.items():
            self.metrics[key].append(value)

    def plot_metrics(self):
        num_metrics = len(self.metrics)
        fig, axes = plt.subplots(1, num_metrics, figsize=(4 * num_metrics, 4))

        for ax, (metric_name, values) in zip(axes, self.metrics.items()):
            ax.plot(values)
            ax.set_title(metric_name.replace("_", " ").title())
            ax.grid(True)

        plt.tight_layout()
        plt.show()


class Trainer:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.metrics = MetricsTracker()

    def create_dataset(self):
        # Generate random points in the feature space
        data = torch.randn(self.config.num_examples, 16) * 2.0

        # Centers much closer together in a smaller square
        centers = torch.tensor(
            [
                [0.7, 0.7],  # Class 0 center
                [-0.7, 0.7],  # Class 1 center
                [-0.7, -0.7],  # Class 2 center
                [0.7, -0.7],  # Class 3 center
            ],
            dtype=torch.float,
        )
        # Much bigger radius to force lots of overlaps
        radii = torch.ones(self.config.num_classes) * 1.8

        # For each point, determine which circles it falls into
        distances = torch.cdist(data[:, :2], centers)
        labels = (distances <= radii).float()

        # Let's verify we got all types of overlaps before proceeding
        num_labels = labels.sum(dim=1)
        for i in range(self.config.num_classes + 1):
            count = (num_labels == i).sum().item()
            print(f"Points in {i} regions: {count}")

        # Create stratified split indices
        all_indices = torch.arange(len(data))
        holdout_indices = []

        # Sample proportionally from each label count group
        for n in torch.unique(num_labels):
            group_indices = torch.where(num_labels == n)[0]
            group_holdout_size = int(0.1 * len(group_indices))
            holdout_indices.extend(
                group_indices[
                    torch.randperm(len(group_indices))[:group_holdout_size]
                ].tolist()
            )

        holdout_indices = torch.tensor(holdout_indices)
        train_indices = torch.tensor(
            [i for i in all_indices if i not in holdout_indices]
        )

        # Create datasets using indices
        train_dataset = TensorDataset(data[train_indices], labels[train_indices])
        holdout_dataset = TensorDataset(data[holdout_indices], labels[holdout_indices])

        train_loader = DataLoader(
            train_dataset, batch_size=self.config.batch_size, shuffle=True
        )
        holdout_loader = DataLoader(
            holdout_dataset, batch_size=len(holdout_indices), shuffle=False
        )

        return train_loader, holdout_loader

    def create_model(self):
        return nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, self.config.num_classes),
        ).to(self.config.device)

    def compute_losses(
        self, output: torch.Tensor, target: torch.Tensor
    ) -> Tuple[torch.Tensor, Dict[str, float]]:
        # BCE loss for set membership probabilities
        bce_loss = F.binary_cross_entropy(output, target)

        # Sparsity loss - encourage fewer set memberships overall
        sparsity_loss = self.config.sparsity_coeff * torch.mean(output)

        # For metrics, we care about set membership accuracy
        with torch.no_grad():
            high_match = (torch.abs(output - target) < 0.2).float().mean().item()
            exact_match = (
                (torch.abs(output - target) < 0.2).all(dim=1).float().mean().item()
            )

        total_loss = bce_loss + sparsity_loss

        metrics = {
            "total_loss": total_loss.item(),
            "bce_loss": bce_loss.item(),
            "sparsity_loss": sparsity_loss.item(),
            "high_match": high_match,
            "exact_match": exact_match,
        }

        return total_loss, metrics

    def _forward(self, network: nn.Module, inputs: torch.Tensor):
        if network.training:
            temp = self.temp_scheduler.step()
        else:
            temp = self.temp_scheduler.value
        output = torch.sigmoid(network(inputs.to(self.config.device)) / temp)
        return output

    def train_step(
        self,
        batch_data: torch.Tensor,
        batch_preds: torch.Tensor,
        network: torch.nn.Module,
        optimizer: torch.optim.Optimizer,
    ) -> Dict[str, float]:
        temp = self.temp_scheduler.step()
        optimizer.zero_grad()

        output = self._forward(network, batch_data)
        loss, metrics = self.compute_losses(output, batch_preds.to(self.config.device))

        loss.backward()
        optimizer.step()

        metrics["temperature"] = temp
        return metrics

    def plot_output_vectors(
        self, network: torch.nn.Module, holdout_loader: torch.utils.data.DataLoader
    ):
        # Get all holdout data
        holdout_data, holdout_labels = next(iter(holdout_loader))

        with torch.no_grad():
            # Count how many regions each point belongs to
            num_labels = holdout_labels.sum(dim=1)
            outputs = self._forward(network, holdout_data)

            # Create subplots - one for each possible number of overlapping regions (0 to num_classes)
            fig, axes = plt.subplots(
                1,
                self.config.num_classes + 1,
                figsize=(4 * (self.config.num_classes + 1), 4),
            )

            # Plot one example for each possible number of overlaps
            for num_regions in range(self.config.num_classes + 1):
                ax = axes[num_regions]
                examples = torch.where(num_labels == num_regions)[0]

                if len(examples) > 0:
                    idx = examples[0]  # Take first example of this type
                    # Plot vector components
                    x = np.arange(len(outputs[idx]))
                    ax.bar(x, outputs[idx].cpu().numpy())

                    # Show which regions this point belongs to
                    active_classes = (
                        torch.where(holdout_labels[idx] > 0)[0].cpu().numpy()
                    )
                    classes_str = (
                        f"Regions: {', '.join(str(c) for c in active_classes)}"
                        if len(active_classes) > 0
                        else "No regions"
                    )

                    ax.set_title(
                        f"In {num_regions} region{'s' if num_regions != 1 else ''}\n{classes_str}"
                    )
                else:
                    ax.set_title(f"No points found\nin {num_regions} regions")

                ax.set_ylim(0, 1)
                ax.set_xticks(x)
                ax.set_xlabel("Vector Component")
                ax.set_ylabel("Magnitude" if num_regions == 0 else "")

        plt.tight_layout()
        plt.show()

    def train(self):
        train_loader, holdout_loader = self.create_dataset()
        network = self.create_model()
        optimizer = torch.optim.SGD(network.parameters(), lr=self.config.learning_rate)

        self.temp_scheduler = CosineScheduler(
            self.config.temp_start,
            self.config.temp_end,
            len(train_loader) * self.config.epochs,
        )

        for epoch in range(self.config.epochs):
            epoch_metrics = defaultdict(list)

            for i, (batch_data, batch_preds) in enumerate(train_loader):
                step_metrics = self.train_step(
                    batch_data, batch_preds, network, optimizer
                )

                for key, value in step_metrics.items():
                    epoch_metrics[key].append(value)

                if i % 100 == 0:
                    metrics_str = [f"Epoch {epoch}, Step {i}"]
                    for key, value in step_metrics.items():
                        if isinstance(value, (int, float)):
                            metrics_str.append(f"{key}: {value:.4f}")
                        else:
                            metrics_str.append(f"{key}: {value}")
                    print(", ".join(metrics_str))

            # Average metrics for the epoch
            avg_metrics = {k: np.mean(v) for k, v in epoch_metrics.items()}
            self.metrics.update_epoch_metrics(avg_metrics)

        self.metrics.plot_metrics()
        self.plot_output_vectors(network, holdout_loader)
        return network, self.metrics.metrics

In [ ]:
config = TrainingConfig(
    sparsity_coeff=0.0,
    epochs=100,
    num_classes=4,
    device="mps",
    learning_rate=1e-2,
    temp_start=1.0,
    temp_end=1e-2
)
trainer = Trainer(config)
model, metrics = trainer.train()

In [34]:
@dataclass
class ReconstructionConfig(TrainingConfig):
    input_dim: int = 128
    latent_dim: int = 16
    double_well_coeff: float = 0.1


class AutoencoderModel(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),  # Much narrower
            nn.ReLU(),
            nn.Linear(32, latent_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim),
        )

    def forward(self, x, temp=1.0):
        h = self.encoder(x)
        z = torch.sigmoid(h / temp)
        out = self.decoder(z)
        return out, z


class ReconstructionTrainer(Trainer):
    def create_dataset(self):
        # Generate random noise
        data = torch.randn(10000, self.config.input_dim)

        # Normalize each vector to unit norm
        data = F.normalize(data, p=2, dim=1)

        # Create train/holdout splits
        indices = torch.randperm(len(data))
        split = int(0.9 * len(data))

        train_dataset = TensorDataset(data[indices[:split]])
        holdout_dataset = TensorDataset(data[indices[split:]])

        train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
        holdout_loader = DataLoader(
            holdout_dataset, batch_size=len(holdout_dataset), shuffle=False
        )

        return train_loader, holdout_loader

    def create_model(self):
        return AutoencoderModel(
            input_dim=self.config.input_dim, latent_dim=self.config.latent_dim
        ).to(self.config.device)

    def compute_losses(self, output, target, latent):
        recon_loss = F.mse_loss(output, target)
        well_loss = self.config.double_well_coeff * torch.mean(latent * (1 - latent))

        total_loss = recon_loss + well_loss

        metrics = {
            "total_loss": total_loss.item(),
            "recon_loss": recon_loss.item(),
            "well_loss": well_loss.item(),
            "sparsity": ((latent < 0.1).float().mean()).item(),
            "binary_ratio": ((latent < 0.1).float() + (latent > 0.9).float())
            .mean()
            .item(),
        }

        return total_loss, metrics

    def train_step(self, batch_data, network, optimizer):
        optimizer.zero_grad()

        output, latent = network(
            batch_data[0].to(self.config.device), self.temp_scheduler.step()
        )

        loss, metrics = self.compute_losses(
            output, batch_data[0].to(self.config.device), latent
        )

        loss.backward()
        optimizer.step()

        metrics["temperature"] = self.temp_scheduler.value
        return metrics

    def plot_output_vectors(self, network, holdout_loader):
        data = next(iter(holdout_loader))[0]
        with torch.no_grad():
            recon, latent = network(
                data.to(self.config.device), self.temp_scheduler.value
            )

        plt.figure(figsize=(15, 5))

        # Plot latent code distributions
        plt.subplot(131)
        plt.hist(latent.cpu().numpy().flatten(), bins=50)
        plt.title("Latent Code Distribution")
        plt.xlabel("Latent Value")
        plt.ylabel("Count")

        # Plot average activation per latent dimension
        plt.subplot(132)
        mean_activations = latent.mean(0).cpu()
        plt.bar(range(len(mean_activations)), mean_activations)
        plt.title("Average Activation Per Latent Dim")
        plt.xlabel("Latent Dimension")
        plt.ylabel("Mean Activation")

        # Plot latent codes for a few examples
        plt.subplot(133)
        num_examples = 10
        plt.imshow(latent[:num_examples].cpu().T, aspect="auto")
        plt.title(f"Latent Codes (first {num_examples} examples)")
        plt.xlabel("Example")
        plt.ylabel("Latent Dimension")
        plt.colorbar()

        plt.tight_layout()
        plt.show()

    def train(self):
        train_loader, holdout_loader = self.create_dataset()
        network = self.create_model()
        optimizer = torch.optim.Adam(network.parameters(), lr=self.config.learning_rate)

        self.temp_scheduler = CosineScheduler(
            self.config.temp_start,
            self.config.temp_end,
            len(train_loader) * self.config.epochs,
        )

        for epoch in range(self.config.epochs):
            epoch_metrics = defaultdict(list)

            for i, batch_data in enumerate(train_loader):
                step_metrics = self.train_step(batch_data, network, optimizer)

                for key, value in step_metrics.items():
                    epoch_metrics[key].append(value)

                if i % 100 == 0:
                    metrics_str = [f"Epoch {epoch}, Step {i}"]
                    for key, value in step_metrics.items():
                        metrics_str.append(f"{key}: {value:.4f}")
                    print(", ".join(metrics_str))

            avg_metrics = {k: np.mean(v) for k, v in epoch_metrics.items()}
            self.metrics.update_epoch_metrics(avg_metrics)

        self.metrics.plot_metrics()
        self.plot_output_vectors(network, holdout_loader)
        return network, self.metrics.metrics

In [ ]:
config = ReconstructionConfig(
    input_dim=128,
    latent_dim=4,
    double_well_coeff=1.0,
    epochs=50,
    device="mps",
    learning_rate=1e-3,
    temp_start=1.0,
    temp_end=1.0,
)
trainer = ReconstructionTrainer(config)
model, metrics = trainer.train()

In [ ]:
import pyvene as pv
import torch

_, tokenizer, gpt2 = pv.create_gpt2()

config = pv.IntervenableConfig(
    [
        {
            "layer": 8,
            "component": "block_output",
            "intervention_type": pv.VanillaIntervention,
        },
        {
            "component": "wte.output",
            "intervention_type": pv.CollectIntervention,
        },
    ]
)

pv_gpt2 = pv.IntervenableModel(config, model=gpt2)

base, counterfactual = pv_gpt2(
    base=tokenizer("The capital of Spain is", return_tensors="pt"),
    sources=[tokenizer("The capital of Italy is", return_tensors="pt"), None],
    # sources=[tokenizer("The capital of Italy is", return_tensors="pt")],
    unit_locations={"sources->base": 3},
)

In [ ]:
base[1][0].shape